In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.05
optimise_mu = [False,True, False, True]
exact = [False, False, True, True]
mc_samples = 100

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46369 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             optimise_mu = optimise_mu[i], exact = exact[i],
                             mc_samples = mc_samples)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-5607aa8a9947a15aca201d7ec782da7c>,
 <Future: pending, key: main-c5ae2182c70ece1773ba499c4fc2df25>,
 <Future: pending, key: main-06ced365ce7063fd3aa1db60efbbe569>,
 <Future: pending, key: main-bffa040b35b9288f880ce05159a308a7>,
 <Future: pending, key: main-ef66c17e6d54b85a0f4962545b982eee>,
 <Future: pending, key: main-a4629726308ce4220b13a8ef69f14759>,
 <Future: pending, key: main-e6b3dd23f1f02a5e57c5642573a75e74>,
 <Future: pending, key: main-60aa9117195e1070b0053db64ee85c51>,
 <Future: pending, key: main-7b75682ac76e509f46c2ab5074aa256e>,
 <Future: pending, key: main-5dbac1fe3275fdb2cc1bc2cdc4c02ea1>,
 <Future: pending, key: main-087f81b71774c0c5aaa9a5071111b874>,
 <Future: pending, key: main-23c0c89df6d5309aaea04142e3893536>,
 <Future: pending, key: main-4160cc93f31124a735b40afff75555d2>,
 <Future: pending, key: main-35bb71030389da24ac7c7b0b19317f55>,
 <Future: pending, key: main-4f83d1678cc4ced010f930ca7e90d96b>,
 <Future: pending, key: main-87b23da5786

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))